<a href="https://colab.research.google.com/github/DGuilherme/MealPlanner/blob/main/MealPlanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd


In [26]:
data = {
    'food_id': range(1, 21),
    'description': [
        'Grilled Chicken Salad',
        'Spaghetti Bolognese',
        'Vegetable Stir-Fry',
        'Cheeseburger',
        'Greek Yogurt with Berries',
        'Steak with Mashed Potatoes',
        'Vegetable Curry',
        'Sushi Platter',
        'Caesar Salad',
        'Pizza Margherita',
        'Shrimp Scampi',
        'Tofu Stir-Fry',
        'Pasta Primavera',
        'Eggplant Parmesan',
        'Beef Tacos',
        'Lobster Bisque',
        'Caprese Salad',
        'Chicken Quesadilla',
        'Beef and Broccoli',
        'Veggie Burger',
    ],
    'food_type': [
        'salad', 'pasta', 'stir-fry', 'burger', 'dessert', 'steak', 'stir-fry', 'sushi', 'salad', 'pizza',
        'seafood', 'stir-fry', 'pasta', 'vegetarian', 'tacos', 'soup', 'salad', 'quesadilla', 'stir-fry', 'burger',
    ],
    'calories': [350, 600, 400, 550, 200, 750, 450, 600, 350, 800, 400, 350, 500, 400, 450, 300, 350, 400, 450, 500],
    'protein (g)': [30, 20, 15, 25, 10, 35, 18, 20, 25, 15, 20, 15, 18, 12, 22, 20, 15, 18, 22, 25],
    'carbohydrates (g)': [10, 50, 30, 40, 25, 40, 35, 45, 20, 70, 30, 30, 60, 40, 25, 20, 15, 35, 30, 60],
    'fat (g)': [15, 10, 8, 30, 5, 40, 15, 12, 20, 20, 15, 10, 15, 25, 15, 10, 10, 20, 15, 25],
}

food_data = pd.DataFrame(data)

# To use the food item descriptions for content-based recommendation
food_indices = pd.Series(food_data.index, index=food_data['description'])

In [34]:
# 4 opçoes para uma semana ou uma opçao para um mes #

# Create a TF-IDF vectorizer for food descriptions
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(food_data['description'])

# Compute the cosine similarity between food items
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def get_recommendations_monthly(num_recommendations=14, cosine_sim=cosine_sim, food_data=food_data):
    # Initialize an empty list to store recommended food items
    recommended_food_items = []

    # Initialize a set to keep track of food types already recommended
    recommended_food_types = set()

    # Loop through all food items and calculate their average similarity to other items
    for idx in range(len(food_data)):
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Exclude the first item (itself)
        sim_scores = sim_scores[1:num_recommendations + 1]

        # Get the indices of recommended food items
        recommended_food_indices = [i[0] for i in sim_scores]

        # Filter out food items that have the same type as already recommended
        filtered_recommendations = [
            food for food in food_data['description'].iloc[recommended_food_indices].to_list()
            if food_data['food_type'].iloc[recommended_food_indices[0]] not in recommended_food_types
        ]

        # If there are any filtered recommendations, add them to the final list
        if filtered_recommendations:
            recommended_food_items.append(filtered_recommendations)

            # Add the food type of the recommended item to the set
            recommended_food_types.add(food_data['food_type'].iloc[recommended_food_indices[0]])

    return recommended_food_items


In [35]:
# so uma opçao para a semana


def get_recommendations_weekly(num_recommendations=14, cosine_sim=cosine_sim, food_data=food_data):
    # Initialize an empty list to store recommended food items
    recommended_food_items = []

    # Create a set to keep track of recommended food types
    recommended_food_types = set()

    # Calculate the combined similarity scores for all food items
    combined_scores = cosine_sim.sum(axis=0)

    # Sort food items based on their combined similarity scores
    sorted_indices = combined_scores.argsort()[::-1]

    # Loop through the sorted indices to get recommendations
    for idx in sorted_indices:
        if food_data['food_type'].iloc[idx] not in recommended_food_types:
            recommended_food_items.append(food_data['description'].iloc[idx])
            recommended_food_types.add(food_data['food_type'].iloc[idx])

        # If we have enough recommendations, stop
        if len(recommended_food_items) >= num_recommendations:
            break

    return recommended_food_items

In [36]:
get_recommendations_weekly()


['Vegetable Stir-Fry',
 'Grilled Chicken Salad',
 'Beef Tacos',
 'Chicken Quesadilla',
 'Cheeseburger',
 'Greek Yogurt with Berries',
 'Steak with Mashed Potatoes',
 'Spaghetti Bolognese',
 'Sushi Platter',
 'Shrimp Scampi',
 'Eggplant Parmesan',
 'Lobster Bisque',
 'Pizza Margherita']

In [37]:
get_recommendations_monthly()

[['Chicken Quesadilla',
  'Caesar Salad',
  'Caprese Salad',
  'Spaghetti Bolognese',
  'Vegetable Stir-Fry',
  'Cheeseburger',
  'Greek Yogurt with Berries',
  'Steak with Mashed Potatoes',
  'Vegetable Curry',
  'Sushi Platter',
  'Pizza Margherita',
  'Shrimp Scampi',
  'Tofu Stir-Fry',
  'Pasta Primavera'],
 ['Grilled Chicken Salad',
  'Vegetable Stir-Fry',
  'Cheeseburger',
  'Greek Yogurt with Berries',
  'Steak with Mashed Potatoes',
  'Vegetable Curry',
  'Sushi Platter',
  'Caesar Salad',
  'Pizza Margherita',
  'Shrimp Scampi',
  'Tofu Stir-Fry',
  'Pasta Primavera',
  'Eggplant Parmesan',
  'Beef Tacos'],
 ['Tofu Stir-Fry',
  'Vegetable Curry',
  'Grilled Chicken Salad',
  'Spaghetti Bolognese',
  'Cheeseburger',
  'Greek Yogurt with Berries',
  'Steak with Mashed Potatoes',
  'Sushi Platter',
  'Caesar Salad',
  'Pizza Margherita',
  'Shrimp Scampi',
  'Pasta Primavera',
  'Eggplant Parmesan',
  'Beef Tacos'],
 ['Beef Tacos',
  'Grilled Chicken Salad',
  'Spaghetti Bolognes